In [1]:
#  các thư viện
# pip install -r requirements.txt

In [69]:
import os
import sys
import json
import argparse
# from collectionamespace import defaultdict, Counter, deque
import mwparserfromhell  
import xml.etree.ElementTree as ET
from tqdm import tqdm
import re

In [3]:
# TẬP HẠT GIỐNG
SEED_LIST_FILE = "seed_list.txt"

XML_FILE = "viwiki-latest-pages-articles.xml"

# ĐỌC FILE TẬP HẠT GIỐNG

## read_seeds

In [4]:
def read_seeds(path):
    '''Đọc danh sách từ file, mỗi dòng là 1 trang wikipedia.'''
    seed_list = []
    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:  # Bỏ qua dòng trống
                seed_list.append(line)
    return seed_list

# test

print(read_seeds(SEED_LIST_FILE))

['Trấn Thành', 'Ninh Dương Lan Ngọc', 'Ngô Thanh Vân', 'Hồng Đào', 'Kiều Minh Tuấn', 'Victor Vũ', 'Charlie Nguyễn', 'Vũ Ngọc Đãng', 'Kaity Nguyễn', 'Jun Vũ']


# ĐỌC TEXT TỪ FILE XML WIKIPEDIA

In [5]:
#test, 30 line
def show_xml_content(xml_path, n_lines=30):
    """
    In ra n_lines dòng đầu của file XML để xem cấu trúc.
    """
    with open(xml_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            print(line.rstrip())
            if i >= n_lines - 1:
                break


show_xml_content("viwiki-latest-pages-articles.xml", n_lines=40)


<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.11/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.11/ http://www.mediawiki.org/xml/export-0.11.xsd" version="0.11" xml:lang="vi">
  <siteinfo>
    <sitename>Wikipedia</sitename>
    <dbname>viwiki</dbname>
    <base>https://vi.wikipedia.org/wiki/Trang_Ch%C3%ADnh</base>
    <generator>MediaWiki 1.45.0-wmf.20</generator>
    <case>first-letter</case>
    <namespaces>
      <namespace key="-2" case="first-letter">Phương tiện</namespace>
      <namespace key="-1" case="first-letter">Đặc biệt</namespace>
      <namespace key="0" case="first-letter" />
      <namespace key="1" case="first-letter">Thảo luận</namespace>
      <namespace key="2" case="first-letter">Thành viên</namespace>
      <namespace key="3" case="first-letter">Thảo luận Thành viên</namespace>
      <namespace key="4" case="first-letter">Wikipedia</namespace>
      <namespace key="5" case="first-letter"

In [6]:
# đọc file XML wiki dump 

# mỗi bài viết trong thẻ <page> </page> 

    # Trả về danh sách [(title, text), ...]

# dùng iterparse (đọc từng phần nhỏ)


def read_pages_from_xml(xml_path):
    """
    Đọc từng trang trong file XML (Wikipedia dump).
    Trả về danh sách [(title, text), ...]
    """
    # danh sách lưu kết quả
    pages = []  
    # iterparse => đọc
    context = ET.iterparse(xml_path, events=("start", "end"))

    # Lấy namespace (ở thẻ <mediawiki>)
    event, root = next(context)
    
    if root.tag.startswith("{"):
        namespace = root.tag.split("}")[0].strip("{")
        namespace_prefix = f"{{{namespace}}}"
    else:
        namespace_prefix = ""

    
    # Duyệt từng p.tử trg XML
    for event, element in context:
        if event == "end" and element.tag == namespace_prefix + "page":
            title_element = element.find(f"./{namespace_prefix}title")
            text_element = element.find(f".//{namespace_prefix}text")
            
            # Lấy nd trg các thẻ, nếu k có thì None 
            if title_element is not None:
                title = title_element.text
            else:
                title = None

            if text_element is not None:
                text = text_element.text
            else:
                text = ''

            pages.append((title, text))  # thêm vào danh sách

            element.clear()  # xóa khỏi bộ nhớ

    return pages



In [7]:
# test in 3 bai
for i, (title, text) in enumerate(read_pages_from_xml(XML_FILE)):
    print(f"{i+1}. {title}")
    print(text[:300])  # chỉ in 300 ký tự đầu
    print("===============")
    if i == 2:  # dừng sau 3 bài đầu
        break


1. Alexandre De Rhodes
#đổi [[Alexandre de Rhodes]]
2. Trang Chính
<templatestyles src="Wiki2021/styles.css" />__NOTOC____NOEDITSECTION__
<div class="mainpage">
{{Wiki2021/Đầu trang}}
<div class="main-body">
    <div class="main-body__left">
{{Wiki2021/Khung
| id = feature-article
| explore card = tfa
| icon modifier = -icon-star
| icon box color modifier = -icon-b
3. Internet Society
{{đổi hướng đến đây|ISOC}}
{{Hộp thông tin tổ chức
| name            = Internet Society<br/>ISOC
| logo            = Internet Society logo.svg
| alt             = Biểu trưng Internet Society
| abbreviation    = ISOC
| founders        = [[Vint Cerf]], [[Bob Kahn]]
| tax_id          = 54-1650477<ref n


In [8]:
all_pages = read_pages_from_xml(XML_FILE)

wiki_dict = {}
for title, text in all_pages:
    if title:
        wiki_dict[title] = text

def get_wikitext(title):
    """
    Lấy nội dung wikitext từ title
    k tìm thấy, trả về chuỗi rỗng
    """
    if title not in wiki_dict: return ""
    
    wikitext = wiki_dict[title]
    if wikitext.startswith("#đổi") or wikitext.startswith("#redirect"):
        start = wikitext.find('[[')
        end = wikitext.find(']]')
        if start != -1 and end != -1:
            new_title = wikitext[start+2:end].strip()
            return get_wikitext(new_title)

    return wikitext


## generate_date_filename

In [9]:
from datetime import datetime

def generate_date_filename():
    now = datetime.now()
    filename = now.strftime("%d-%m-%Y_%H-%M-%S")
    return filename

print(generate_date_filename())


25-10-2025_09-58-51


In [10]:
#test1
print(get_wikitext("Ninh Dương Lan Ngọc"))

# with open("test/test_wikitext.txt", "w", encoding="utf-8") as f:
#     f.write(get_wikitext("Ninh Dương Lan Ngọc"))

{{Thông tin nhân vật
| tên = Ninh Dương Lan Ngọc
| hình = Ninh Dương Lan Ngọc trong Mối Tình Đầu Của Tôi.png
| ghi chú hình = Ninh Dương Lan Ngọc vào năm 2019
| tên khai sinh = 
| ngày sinh = {{Birth date and age|1990|4|4}}
| quốc tịch = {{VIE}}
| học vấn = [[Trường Đại học Sân khấu – Điện ảnh Thành phố Hồ Chí Minh]]
| công việc = {{hlist|[[Diễn viên]]}}
| năm hoạt động = 2009–nay
| con cái = 
| giải thưởng = [[#Giải thưởng và đề cử|Danh sách]]
| nơi sinh = [[Thành phố Hồ Chí Minh]], [[Việt Nam]]
| tên khác = 
| dân tộc = [[Người Việt|Kinh]]
| quê quán = [[Nam Định]], [[Việt Nam]]
| notable works = [[#Danh sách phim|Danh sách]]
| signature = Signature of Ninh Duong Lan Ngoc.svg
}}

'''Ninh Dương Lan Ngọc''' (sinh ngày 4 tháng 4 năm 1990) là một nữ [[diễn viên]] người Việt Nam. Được đánh giá là một trong những nữ diễn viên Việt Nam xuất sắc nhất trong thế hệ của mình, cô bắt đầu được biết đến qua vai diễn Nương trong bộ phim ''[[Cánh đồng bất tận (phim)|Cánh đồng bất tận]]''.<ref>{{Chú 

## chung

In [286]:
#chung

def save_to_txt(content ,file_name = None, auto_name = False):
    if auto_name or file_name is None:
        file_name = generate_date_filename() 
         
    with open(f"test/{file_name}.txt", "w",  encoding="utf-8") as f:
        f.write(content) 

def save_to_json(content ,file_name = None,auto_name = False):
    if auto_name or file_name is None:
        file_name = generate_date_filename()
    with open(f"test/{file_name}.json", "w",  encoding="utf-8") as f:
        json.dump(content, f, ensure_ascii=False, indent=4)


def print_json_format(res):
    print(json.dumps(res, ensure_ascii=False, indent=4))

def df_to_dict(df, orient='records'):
    return df.to_dict(orient=orient)

# TRÍCH XUẤT THÔNG TIN CƠ BẢN

## extract_infobox_from_wikitext

In [12]:
def extract_infobox_from_wikitext(wikitext):
    
    if not wikitext:
        return {}
    try:
        # biến chuỗi wikitext => Wikicode obj
        code = mwparserfromhell.parse(wikitext)

    except Exception:
        return {}
    
    # duyệt qua toàn bộ cấu trúc wikitext, trả về list all các template (các khối {{ ... }})
    templates = code.filter_templates() 
   
    info = {}

    for template in templates:
        # template là obj

        # tên của template 
        name = str(template.name).lower()
        infoboxs = ["nhân vật", "phim", "diễn viên", "nghệ sĩ",  
                    "film", "truyền hình", "điện ảnh",
                    "television"]

        for infobox in infoboxs:
            if infobox in name.lower():
                # iterate params. in list tham so
                for param in template.params:
                    key = str(param.name).strip()
                    value = str(param.value).strip()
                    if key and value:
                        info[key] = value
                # break nếu k rỗng k false 
                if info:
                    break
    return info

## check_type_of_infobox

In [13]:
def check_type_of_infobox(wikitext):
    if not wikitext:
        return {}
    try:
        # biến chuỗi wikitext => Wikicode object
        code = mwparserfromhell.parse(wikitext)

    except Exception:
        return {}
    
    templates = code.filter_templates() 

    # thông tin nhân vật => nhân vật
    person_keys = [
        "nhân vật", "diễn viên", "nghệ sĩ"
    ]
    film_keys = [
        "phim", 
        "film",
        "truyền hình",
        "điện ảnh",
        'television'
    ]
    for template in templates:
        name = str(template.name).lower()
        for key in film_keys:
            if key in name.lower(): return "film"
        for key in person_keys:
            if key in name.lower(): return "person"
    return ""

In [138]:
#test
ndln_wikitext = get_wikitext("Ninh Dương Lan Ngọc")

print(extract_infobox_from_wikitext(ndln_wikitext))

# save_to_json(extract_infobox_from_wikitext(ndln_wikitext), "test/test_infobox" )

{'tên': 'Ninh Dương Lan Ngọc', 'hình': 'Ninh Dương Lan Ngọc trong Mối Tình Đầu Của Tôi.png', 'ghi chú hình': 'Ninh Dương Lan Ngọc vào năm 2019', 'ngày sinh': '{{Birth date and age|1990|4|4}}', 'quốc tịch': '{{VIE}}', 'học vấn': '[[Trường Đại học Sân khấu – Điện ảnh Thành phố Hồ Chí Minh]]', 'công việc': '{{hlist|[[Diễn viên]]}}', 'năm hoạt động': '2009–nay', 'giải thưởng': '[[#Giải thưởng và đề cử|Danh sách]]', 'nơi sinh': '[[Thành phố Hồ Chí Minh]], [[Việt Nam]]', 'dân tộc': '[[Người Việt|Kinh]]', 'quê quán': '[[Nam Định]], [[Việt Nam]]', 'notable works': '[[#Danh sách phim|Danh sách]]', 'signature': 'Signature of Ninh Duong Lan Ngoc.svg'}


In [15]:
def extract_wikilinks_from_wikitext(text):
   
    if not text:  # Nếu nội dung rỗng thì trả về []
        return []

    try:
        wikicode = mwparserfromhell.parse(text)  # Ptích cú pháp wikitext => obj có thể duyệt
    except Exception:
        return []  # Nếu lỗi parse thì bỏ qua

    links = []  # Danh sách kết quả
    for link in wikicode.filter_wikilinks():  # Lọc ra all [[...]] trong wikitext
        title = str(link.title).strip()       # Lấy phần title trong [[title|...]]
        if title:
            links.append(title)

    return links

In [139]:
#test 
ndln_wikilink = extract_wikilinks_from_wikitext(ndln_wikitext)
print(ndln_wikilink)

# save_to_json(ndln_wikilink,"test/test_link" )

['Trường Đại học Sân khấu – Điện ảnh Thành phố Hồ Chí Minh', 'Diễn viên', '#Giải thưởng và đề cử', 'Thành phố Hồ Chí Minh', 'Việt Nam', 'Người Việt', 'Nam Định', 'Việt Nam', '#Danh sách phim', 'diễn viên', 'Cánh đồng bất tận (phim)', 'Báo Tuổi trẻ', 'Thành phố Hồ Chí Minh', 'Trường Đại học Sân khấu - Điện ảnh Thành phố Hồ Chí Minh', 'VnExpress', 'quảng cáo', 'ca sĩ', 'Gia đình phép thuật', 'Cánh đồng bất tận (phim)', 'Cánh Diều Vàng', 'Ngô Thanh Vân', 'Bước nhảy hoàn vũ', 'Dân Trí', 'Cô Ba Sài Gòn', 'Tấm Cám: Chuyện chưa kể', 'Gái già lắm chiêu 3', 'Cua lại vợ bầu', 'Gái già lắm chiêu 3', 'Chạy đi chờ chi', 'Chị đẹp đạp gió rẽ sóng', 'VTV3', 'Chị đẹp đạp gió rẽ sóng', 'Đạp gió 2024', 'Cao Thái Sơn', 'Only C', 'Đông Nhi', 'Cô Ba Sài Gòn', 'Jun Phạm', 'Cô Ba Sài Gòn', 'S.T Sơn Thạch', 'S.T Sơn Thạch', 'Cô Ba Sài Gòn', 'Tập đoàn LG', 'Rhymastic', 'Rhymastic', 'ZaloPay', 'Only C', 'Only C', 'Liên Quân', 'Lazada Việt Nam', 'S.T Sơn Thạch', 'Trang Pháp', 'Gia đình phép thuật', 'Mệnh lệnh Hoa

In [137]:
#test 

matBiec_wikitext = get_wikitext("Mắt biếc (phim)")
print(matBiec_wikitext)
# save_to_txt(matBiec_wikitext,"test/test_film_wikitext")

matBiec_infobox = extract_infobox_from_wikitext(matBiec_wikitext)
print("type mat biec infobox",type(matBiec_infobox))
# save_to_json( matBiec_infobox,"test/test_film_infobox")

matBiec_wikilink = extract_wikilinks_from_wikitext(matBiec_wikitext)
print("type cua wikilink mat Biec", type(matBiec_wikilink))
# save_to_json( matBiec_wikilink,"test/test_film_wikilink")

{{for|tiểu thuyết cùng tên|Mắt biếc (tiểu thuyết)}}
{{Thông tin phim
| tên = Mắt biếc
| hình = Áp phích phim Mắt biếc.jpg
| caption = Áp phích chiếu rạp của phim tại Việt Nam
| đạo diễn = [[Victor Vũ]]
| sản xuất = {{Liệt kê|
* Nguyễn Quốc Công
* Kay Nguyễn
}}
| kịch bản = {{Liệt kê|
* Victor Vũ
* A Type Machine
}}
| dựa trên = ''[[Mắt biếc (tiểu thuyết)|Mắt biếc]]'' của [[Nguyễn Nhật Ánh]]
| diễn viên = {{Liệt kê|
* [[Trần Nghĩa (diễn viên)|Trần Nghĩa]]
* [[Trúc Anh]]
* Trần Phong
* Khánh Vân
* Thảo Tâm
}}
| âm nhạc = [[Christopher Wong]]<br>[[Phan Mạnh Quỳnh]]
| quay phim = [[Dominic Pereira]]<ref>{{chú thích web|url=https://danviet.vn/nhung-guong-mat-nuoc-ngoai-thanh-cong-voi-dien-anh-viet-20220916084332568.htm|title=Những gương mặt nước ngoài thành công với điện ảnh Việt|author=Thủy Vũ|date=2022-09-16|publisher=[[Dân Việt (báo)|Dân Việt]]|access-date=2024-05-19}}</ref>
| dựng phim = Vũ Hoàng Anh
| studio = [[Galaxy EE|Galaxy M&E]]<br>November Films
| phát hành = [[Galaxy Studio]]
|

Wikipedia không nói rõ đâu là phim, đâu là người
chỉ tiêu đề + cú pháp [[...]]

phân loại liên kết -> người / phim

## is_vn_film_wikilink

In [18]:
# check xem 1 link là phim or k 
def is_vn_film_wikilink(link_title):
    if link_title is None or link_title.strip() == "":
        return False   
     
    # tức là link này k có dấu hiệu nhận biết tại tiêu đề
    # get text ra để check infobox 
    wikitext = get_wikitext(link_title)
    if (check_type_of_infobox(wikitext) == "film"):
        infobox = extract_infobox_from_wikitext(wikitext)
        # gom hết text trong infobox để tìm
        text_all = " ".join(str(v) for v in infobox.values()).lower()
        for x in [ "{{vie}}",'[[tiếng việt]]', 'tiếng việt'] :
            if x in text_all:
                # print(x)
                return True
           
                

            
            
  
    return False

#test 
# is_vn_film_wikilink
# check_type_of_infobox
print(is_vn_film_wikilink("Cô gái từ quá khứ"))
print(is_vn_film_wikilink("Cánh đồng bất tận (phim)"))
print(is_vn_film_wikilink('Kong: Đảo Đầu lâu'))
print(is_vn_film_wikilink('Long Ruồi'))

print(is_vn_film_wikilink('Tèo em'))
print(is_vn_film_wikilink('Tấm Cám: Chuyện chưa kể'))

x = 'Vừa đi vừa khóc'
print(is_vn_film_wikilink(x))



True
True
False
True
True
True
True


In [136]:
#test
print(check_type_of_infobox(x))
x1 = get_wikitext(x)
save_to_txt(x1, f'test/{generate_date_filename()}')
x2 = extract_infobox_from_wikitext(x1)
print(x2)


{'show_name': 'Vừa đi vừa khóc', 'image': 'Vừa đi vừa khóc poster.jpg', 'format': '[[Phim truyền hình]]', 'genre': '[[Hài hước]]<br>[[Tâm lý xã hội]]<br>[[Tình cảm]]', 'writer': '[[Vũ Ngọc Đãng]]', 'director': '[[Vũ Ngọc Đãng]]', 'creative_director': '[[Nguyễn Nam]]<br>Nguyễn Trung', 'starring': '[[Lương Mạnh Hải]]<br>[[Minh Hằng]]<br>[[Nhã Phương (diễn viên)|Nhã Phương]]<br>[[Ninh Dương Lan Ngọc]]<br>[[Thanh Thủy (diễn viên)|Thanh Thủy]]<br>[[Bảo Anh]]<br>La Quốc Hùng', 'theme_music_composer': '[[Minh Thư (ca sĩ)|Minh Thư]]', 'opentheme': '"Vừa đi vừa khóc"', 'endtheme': '"Nắng"', 'country': '{{VNM}}', 'language': '[[Tiếng Việt]]', 'num_episodes': '36', 'cinematography': 'Nguyễn Phương<br>Thanh Phương', 'runtime': '40 - 45 phút/tập (có bao gồm [[quảng cáo]])', 'company': '[[BHD]]<br>TVPlus', 'channel': '[[VTV3]]', 'first_aired': '17 tháng 3 năm 2014', 'last_aired': '7 tháng 7 năm 2014', 'preceded_by': '[[Chỉ có thể là yêu (phim truyền hình Việt Nam)|Chỉ có thể là yêu]]', 'followed_by

## is_vn_person_wikilink

In [20]:
# check 1 link la person or k

def is_vn_person_wikilink(link_title):
    if link_title is None or link_title.strip() == "":
        return False
    
      
    # link này k có dấu hiệu nhận biết tại title
    # get text ra để check infobox 
    wikitext = get_wikitext(link_title)
    if (check_type_of_infobox(wikitext) == "person"):
          
        infobox = extract_infobox_from_wikitext(wikitext)
               
        # gom hết text trong infobox để tìm
        text_all = " ".join(str(v) for v in infobox.values()).lower()
        # print(text_all)
            
        if any(x in text_all for x in ["việt nam", "{{vie}}", "vietnam"]):
            # print("=> Người Việt Nam")
            return True
                
                

 
    return False

#test

print(is_vn_person_wikilink("Ninh Dương Lan Ngọc"))
print(is_vn_person_wikilink("Trần Nghĩa (diễn viên)"))
print(is_vn_person_wikilink("Trúc Anh"))
print(is_vn_person_wikilink("Victor Vũ"))
print(is_vn_person_wikilink("Thanh Hiền"))

True
True
True
True
True


In [21]:
# print(get_wikitext("Trúc Anh"))
print(check_type_of_infobox(get_wikitext("Trúc Anh")))


person


## unique_list

In [22]:
#chung
# Bỏ lặp, giữ đúng thứ tự 

def unique_list(items_list):

    seen_items = set()   # chứa các phần tử đã gặp
    result = []          # danh sách kết quả

    for item in items_list:
        if item not in seen_items:
            result.append(item)    # thêm vào kết quả
            seen_items.add(item)   # đánh dấu là đã gặp

    return result


#test 
print(unique_list(["Trấn Thành", "Ngô Thanh Vân", "Trấn Thành"]))

['Trấn Thành', 'Ngô Thanh Vân']


In [24]:
test_wikilink_list = []
for l in ndln_wikilink:
    if is_vn_film_wikilink(l): 
        print(l)
        test_wikilink_list.append(l)

test_wikilink_list = unique_list(test_wikilink_list)
print(len(test_wikilink_list))

Cánh đồng bất tận (phim)
Gia đình phép thuật
Cánh đồng bất tận (phim)
Bước nhảy hoàn vũ
Cô Ba Sài Gòn
Tấm Cám: Chuyện chưa kể
Gái già lắm chiêu 3
Cua lại vợ bầu
Gái già lắm chiêu 3
Chạy đi chờ chi
Chị đẹp đạp gió rẽ sóng
VTV3
Chị đẹp đạp gió rẽ sóng
Cô Ba Sài Gòn
Cô Ba Sài Gòn
Cô Ba Sài Gòn
Gia đình phép thuật
Mệnh lệnh Hoa hồng
Vừa đi vừa khóc
Nhân tình lạc lối
Nguyệt thực (phim truyền hình)
Cánh đồng bất tận (phim)
Long Ruồi
Tèo em
Trúng số (phim 2015)
Tấm Cám: Chuyện chưa kể
Cô Ba Sài Gòn
Cua lại vợ bầu
Gái già lắm chiêu 3
Gái già lắm chiêu V: Những cuộc đời vương giả
Cô gái từ quá khứ
Bí mật đêm Chủ Nhật
Bước nhảy hoàn vũ (mùa 6)
Bữa trưa vui vẻ
Không giới hạn - Sasuke Việt Nam
Ơn giời cậu đây rồi!
Người bí ẩn
Nhanh như chớp (trò chơi truyền hình)
Người bí ẩn
Vì bạn xứng đáng
Giọng ải giọng ai
7 Nụ cười Xuân
Running Man (chương trình truyền hình Việt Nam)
Người bí ẩn
Người ấy là ai
Nhanh như chớp (trò chơi truyền hình)
7 Nụ cười Xuân
7 Nụ cười Xuân
Running Man Vietnam
7 Nụ cười Xuâ

## parse_heading

In [56]:
# == Danh sách phim == => {'level': 2, 'title': 'Danh sách phim'}
def parse_heading(line):
    line = line.strip() # clear space đầu cuối

    # k '=' ở đầu, cuối => pass
    if not line.startswith('=') or not line.endswith('='):
        return None  # không phải tiêu đề wiki

    # Đếm số '=' ở đầu và cuối
    left_eq = len(line) - len(line.lstrip('='))
    right_eq = len(line) - len(line.rstrip('='))

    if left_eq != right_eq: # ==abc===
        return None  # không cân đối, bỏ qua

    # Cắt phần tên tiêu đề ở giữa
    title = line[left_eq:len(line) - right_eq].strip()
    if not title: return None # k content 

    return {
        'level': left_eq, # số lượng '=' --> cấp độ title
        'title': title
    }

# Thử
lines = [
    "== Danh sách phim ==",
    "=== Điện ảnh ===",
    "==== Truyền hình ====",
    "Không phải tiêu đề",
]

for l in lines:
    result = parse_heading(l)
    if result:
        print(result)


{'level': 2, 'title': 'Danh sách phim'}
{'level': 3, 'title': 'Điện ảnh'}
{'level': 4, 'title': 'Truyền hình'}


## extract_films_section

In [76]:
def extract_films_section(wikitext,depth=0):
    """
    Tách phần wikitext thuộc tiêu đề cấp 2 (== ... ==)
    có 'phim' hoặc 'film' trong tiêu đề.
    Giữ nguyên wikitext gốc, chỉ so khớp tên tiêu đề ở dạng thường.
    Nếu section chỉ chứa liên kết {{chính|...}} thì đệ quy sang trang đó.
    """
    if not wikitext or depth > 2: return ""

    lines = wikitext.splitlines()
    headings = []  # lưu (index_dòng, heading_dict)

    for i, line in enumerate(lines):
        h = parse_heading(line)
        if h:
            headings.append((i, h))

    # chọn heading cấp 2 có từ "phim"/"film"
    chosen = None
    for i, h in headings:
        title_lower = h['title'].lower()
        if h['level'] == 2 and ('danh sách phim' in title_lower or 'phim' in title_lower or 'film' in title_lower):
            chosen = (i, h)
            break
        elif h['level'] == 2 and ('điện ảnh' in title_lower):
            chosen = (i, h)  
            # (51, {'level': 2, 'title': 'Sự nghiệp điện ảnh'})
            break

    if not chosen: return ""

    start_line_index = chosen[0]
    chosen_level = chosen[1]['level'] # 2 => level 2 

    # tìm heading cấp 2 kế tiếp để biết điểm kết thúc
    end_line_index = len(lines)
    for i, h in headings:
        if i > start_line_index and h['level'] == chosen_level:
            end_line_index = i
            break

    # trích phần nội dung giữa hai heading
    section = "\n".join(lines[start_line_index + 1:end_line_index]).strip()
    
    # --- kiểm tra nếu chỉ là liên kết {{chính|...}} hoặc {{main|...}} ---
    main_re = re.search(r"\{\{\s*(?:chính|main)\s*\|\s*([^}|]+)", section, flags=re.IGNORECASE)
    if main_re:
        linked_title = main_re.group(1).strip()
        try:
            linked_wikitext = get_wikitext(linked_title)
            if linked_wikitext:
                # đệ quy sang trang phụ, tăng depth
                return extract_films_section(linked_wikitext, depth=depth + 1)
        except Exception as e:
            print("Lỗi khi lấy wikitext:", e)
    return section

# test
print(extract_films_section(get_wikitext("Trấn Thành")))

=== Phim truyền hình ===
{| class="wikitable" 
|-
!Năm!!Tựa!!Vai
!Chú thích
|-
|2006
|[[Mùi ngò gai]]
|MC
|
|-
|2007 ||''Đời cần có nhau''||Bá 
|
|-
| rowspan="3" |2009 ||''Cô nàng bướng bỉnh''|| Hải Long
|
|-
|''Ngõ vắng''||Vũ
|
|-
|''Sau ánh hào quang''||bác sĩ Vũ
|
|-
| rowspan="4" |2010||''Bước chân hoàn vũ''||MC
|
|-
|[[Cổng mặt trời (phim truyền hình)|Cổng mặt trời]] ||Bạn của Quốc
|
|-
|''Phía sau ánh bình minh''||Việt Hoàn 
|
|-
|''[[Chuyện tình mùa thu]]''||Bảo
|
|- 
| rowspan="3" |2011||''Sự thật vô hình''||Thảo Bình
|
|-
|''[[Một thời ta đuổi bóng]]''||Trí trề
|
|-
|''Chuyện tình làng hoa''||Long té giếng 
|
|-
| rowspan="2" |2012||''Giấc mơ cỏ may''||luật sư Gia Huy 
|
|-
|''Khoảnh khắc tình cờ''||Lãm 
|
|-
|2013 ||''Những thiên thần nhanh nhạy''||MC Trấn Thành 
|
|}

=== Phim điện ảnh ===
{| class="wikitable" 
|-
!Năm!!Tựa phim!!Vai diễn
!Chú thích
|-
|2010||''[[Công chúa teen và ngũ hổ tướng]]''||MC Trấn Thành
|
|-
|2010||''Em hiền như ma sơ''||Trần Hàn Vi
|
|-
|2012||''[

## extract_films_from_subsection

In [85]:
def extract_films_from_subsection(film_section_wikitext):
    if not film_section_wikitext:
        return []

    lines = film_section_wikitext.splitlines()
    headings = []

    # --- Thu thập các heading ---
    for i, line in enumerate(lines):
        h = parse_heading(line)
        if h:
            headings.append((i, h))

    # --- Lọc các heading level 3 ---
    level3_heads = [(i, h) for i, h in headings if h["level"] == 3]
    if not level3_heads:
        print("Section không có heading cấp 3")
        return []

    results = []
    for idx, (start_line, h) in enumerate(level3_heads):
        # xác định end_line là heading kế tiếp hoặc hết file
        end_line = len(lines)
        if idx + 1 < len(level3_heads):
            end_line = level3_heads[idx + 1][0]

        subsection = "\n".join(lines[start_line + 1:end_line]).strip()
        results.append({
            "title": h["title"].strip(),
            "subsection_wikitext": subsection
        })

    return results

#test
b0 ="Trấn Thành"
b1 = "Ninh Dương Lan Ngọc"
b2 = "Ngô Thanh Vân"
a0 = b2
a1 = (extract_films_section(get_wikitext(a0)))
a2= extract_films_from_subsection(a1)
print_json_format(a2)


[
    {
        "title": "Phim điện ảnh",
        "subsection_wikitext": "{| class=\"wikitable\"\n|- style=\"background:#ccc; text-align:center;\"\n! rowspan=\"2\" style=\"width:33px;\"|Năm\n! rowspan=\"2\" | Tựa đề\n! colspan=\"2\" |Vai trò\n! rowspan=\"2\" |Vai diễn\n! rowspan=\"2\" |Ghi chú\n|-\n! Đạo diễn\n! Sản xuất\n|-\n|2001\n|''Hương dẻ''\n| {{Không}}\n| {{Không}}\n|Na lớn\n|rowspan=\"2\" |phim truyền hình\n|-\n|2004\n|''Rouge''\n| {{Không}}\n| {{Không}}\n|Thủy\n|-\n| rowspan=\"3\" | 2006\n|''Ngôi nhà bí ẩn''\n| {{Không}}\n| {{Không}}\n|Đạo diễn Trúc\n|\n|-\n|''2 trong 1''\n| {{Không}}\n| {{Không}}\n|Như Lan\n| \n|-\n|''Chuyện tình Sài Gòn''\n| {{Không}}\n| {{Không}}\n|Tâm\n|2008 chiếu ở Việt Nam\n|-\n|2007\n|''[[Dòng máu anh hùng]]''\n| {{Không}}\n| {{Không}}\n|Võ Thanh Thúy\n|\n|-\n|2009\n|''[[Bẫy rồng]]''\n| {{Không}}\n| {{Không}}\n|Trinh / Phượng hoàng\n|\n|-\n|2010\n|''Biệt đội ưng biển''\n| {{Không}}\n| {{Không}}\n|Nữ cảnh sát đa quốc gia Tú Mi\n|\n|-\n|2011\n|''Khao khát

## extract_headers

In [263]:
def extract_headers(wikitext):
    """
    Trích xuất header hoàn chỉnh từ bảng wikitext (2 dòng header).
    Nếu có ! colspan="n" | HeaderCha thì bỏ HeaderCha, thay bằng n header con dòng dưới.
    """
    wikitext = re.sub(r'style\s*=\s*["\'].*?["\']', '', wikitext)

    lines = wikitext.strip().split('\n')
    header_lines = []
    
    # Lấy tất cả dòng header (bắt đầu bằng !)
    for line in lines:
        line = line.strip()
        if line.startswith('!'):
            header_lines.append(line)
       
    # print('@1__', header_lines)
    if not header_lines:
        return []

    # Xử lý từng dòng thành list các ô
    headers_res = []
    headers_row2 = []
    header_rowspan = False # mặc định header 1 dòng thôi, True là 2 dòng 

    if "rowspan" in header_lines[0]:
        header_rowspan = True
    
    hash_index = None
    for line in header_lines:
        if header_rowspan:
            # print('@2__',line) #! rowspan="2" |Năm
            # bỏ ! đi đã 
            line = line.replace("!",'').strip()
            if line.startswith('rowspan'):
                parts = line.split('|')
                # ['rowspan="2" ', 'Năm']
                headers_res.append(parts[1].strip())
                continue
            elif line.startswith('colspan'):
                headers_res.append('#')
                continue
            else:
                headers_row2.append(line.strip())

        else: 
            if line.startswith("!"):
                line = line[1:] # xóa kí tự đầu 
                columns = line.split('!!')
                for c in columns: headers_res.append(c)
                continue # add xong thì bỏ qa    
            
    for i, item in enumerate(headers_res):
        if item == '#':
            hash_index = i
            break   # nếu chỉ muốn vị trí đầu tiên của '#'
            
    if hash_index is not None:
        headers_res[hash_index:hash_index] = headers_row2 # chèn trước #
        headers_res.remove('#')  # xóa ký tự '#'

    return(headers_res)


#test
wikitext1 = """
|- style="background:#ccc; text-align:center;" 
! rowspan="2" style="width:33px;"|Năm
! rowspan="2" | Tựa đề
! colspan="2" |Vai trò
! rowspan="2" |Vai diễn
! rowspan="2" |Ghi chú
|-
! Đạo diễn
! Sản xuất
|-
"""

wikitext = '''
{| class="wikitable"
!Năm!!Tựa phim!!Vai diễn!!Ghi chú
|-
'''
print(extract_headers(wikitext1))
print(extract_headers(wikitext))

print(extract_headers(a2[0]['subsection_wikitext']))

['Năm', 'Tựa đề', 'Đạo diễn', 'Sản xuất', 'Vai diễn', 'Ghi chú']
['Năm', 'Tựa phim', 'Vai diễn', 'Ghi chú']
['Năm', 'Tựa đề', 'Đạo diễn', 'Sản xuất', 'Vai diễn', 'Ghi chú']


In [250]:
def clean_wiki_markup(text):
    code = mwparserfromhell.parse(text)

    # Nếu có wikilink [[...]]
    links = code.filter_wikilinks()
    if links:
        return str(links[0].title).strip()

    # Nếu có template {{...}}
    templates = code.filter_templates()
    if templates:
        return str(templates[0].name).strip()

    # Nếu không có gì đặc biệt
    return code.strip_code().strip()

print(clean_wiki_markup("[[Ngô Thanh Vân]]"))
print(clean_wiki_markup('{{Không}}'))
print(clean_wiki_markup("''ABC''"))

Ngô Thanh Vân
Không
ABC


In [271]:
def parse_span(token):
    """
    Nhận 1 token dạng 'rowspan="2" | dữ liệu' hoặc chỉ dữ liệu
    Trả về (rowspan, colspan, data)
    """
    rowspan = 1
    colspan = 1
    data = token.strip()
    
    # Kiểm tra xem có rowspan/colspan không
    parts = data.split('|', 1)  # split 1 lần, còn lại là data
    attr = parts[0].strip()
    
    # check rowspan
    if 'rowspan' in attr:
        try:
            # loại bỏ ký tự thừa
            value = ''.join(c for c in attr if c.isdigit())
            rowspan = int(value)
        except:
            pass
        data = parts[1].strip() if len(parts) > 1 else ''
    
    # check colspan
    if 'colspan' in attr:
        try:
            value = ''.join(c for c in attr if c.isdigit())
            colspan = int(value)
        except:
            pass
        data = parts[1].strip() if len(parts) > 1 else ''
    
    return rowspan, colspan, data

# Test
lines = [
    '|2001',
    "|''Hương dẻ''",
    '| {{Không}}',
    '| {{Không}}',
    '|Na lớn',
    '|rowspan="2" |phim truyền hình',
    '|2004',
    "|''Rouge''",
]

for line in lines:
    if line.startswith('|'):
        r, c, d = parse_span(line[1:])
        print(r, c, d)

1 1 2001
1 1 ''Hương dẻ''
1 1 {{Không}}
1 1 {{Không}}
1 1 Na lớn
2 1 phim truyền hình
1 1 2004
1 1 ''Rouge''


In [297]:
#split_top_level_double tách dòng theo || nhưng bỏ qua những || nằm trong [[...]] hoặc {{...}}
# split theo ||
def split_theo_gach_kep(s):
    """
    Split string s on top-level '||' occurrences (ignore those inside [[...]] or {{...}}).
    Returns list of segments.
    """
    segs = []
    buf = []
    i = 0
    n = len(s)
    # nesting counters for [[...]] and {{...}} and also [ and ] and { and }
    brack = 0
    curl = 0
    squote = 0
    while i < n:
        # check for '[[' or ']]', '{{' or '}}'
        if s.startswith('[[', i):
            brack += 1
            buf.append('[[')
            i += 2
            continue
        if s.startswith(']]', i):
            if brack > 0: brack -= 1
            buf.append(']]'); i += 2; continue
        if s.startswith('{{', i):
            curl += 1
            buf.append('{{'); i += 2; continue
        if s.startswith('}}', i):
            if curl > 0: curl -= 1
            buf.append('}}'); i += 2; continue

        # top-level '||' when not inside brackets/templates
        if s.startswith('||', i) and brack == 0 and curl == 0:
            segs.append(''.join(buf).strip())
            buf = []
            i += 2
            continue

        buf.append(s[i])
        i += 1

    segs.append(''.join(buf).strip())
    return segs

def find_top_level_pipe(s):
    """
    Find index of first top-level '|' used as attribute/value separator,
    ignoring '|' inside [[...]] or {{...}}.
    Returns index or -1 if none.
    """
    i = 0
    n = len(s)
    brack = 0
    curl = 0
    while i < n:
        if s.startswith('[[', i):
            brack += 1; i += 2; continue
        if s.startswith(']]', i):
            if brack > 0: brack -= 1
            i += 2; continue
        if s.startswith('{{', i):
            curl += 1; i += 2; continue
        if s.startswith('}}', i):
            if curl > 0: curl -= 1
            i += 2; continue

        # single '|' char (not '||' because those were handled earlier)
        if s[i] == '|' and brack == 0 and curl == 0:
            return i
        i += 1
    return -1

a5 ='[[abc||cvd]]||abc||efd|b'
print(split_theo_gach_kep(a5))

['[[abc||cvd]]', 'abc', 'efd|b']


## wikitable_to_df

In [306]:
import mwparserfromhell
import re
import pandas as pd

import re


def wikitable_to_df(wikitext, headers):
    lines = wikitext.strip().split('\n')
    
    # số cột
    num_cols = len(headers)

    table_rows = []          # danh sách các dòng đã parse
    rowspan_tracker = {}     # theo dõi các ô có rowspan còn kéo dài
    
    current_row_index = -1        # chỉ số hàng đang xử lý
    for line in lines:
        line = line.strip()
      
        # {| class wikitable or open table => pass
        if not line or(line.startswith("{|")): continue
        # end of table
        if line.startswith("|}") or line == "}" or line == "|}" :
            break
        # ---Thêm dòng mới trống:
        # ---đại diện cho 1 hàng dữ liệu mới sắp được điền.
        if line.startswith('|-'):
            table_rows.append([None]*num_cols) #grid = [[None, None, None]]
            current_row_index += 1 #0
            # --- Áp dụng các ô rowspan còn hiệu lực cho dòng hiện tại ---
            for key in list(rowspan_tracker.keys()):
                start_row, col = key
                remaining_rows, value = rowspan_tracker[key]

                # Nếu dòng hiện tại nằm sau dòng bắt đầu rowspan
                if current_row_index > start_row and remaining_rows > 0:
                    # Nếu ô ở vị trí này đang trống thì điền giá trị
                    if table_rows[current_row_index][col] is None:
                        table_rows[current_row_index][col] = value

                    # Giảm số dòng còn lại của rowspan đi 1
                    remaining_rows -= 1
                    if remaining_rows <= 0: del rowspan_tracker[key]
                    else: rowspan_tracker[key] = (remaining_rows, value)

            continue
        
        if line.startswith('!'): continue # pass
        # dòng dữ liệu bình thường bắt đầu bằng '|' 
        if (line.startswith('|')):
            # đảm bảo có một hàng hiện tại; else hãy tạo một hàng (xử lý các bảng bỏ qua '|-' ban đầu) 
            if current_row_index == -1: 
                table_rows.append([None] * num_cols) 
                current_row_index = 0
            # xóa | ở đầu dòng đi
            line = line[1:].strip() 
            # chia thành các phân đoạn '||'
            segments = split_theo_gach_kep(line)

            # --- Tìm vị trí cột trống đầu tiên trong hàng hiện tại ---
            row = table_rows[current_row_index]
            # tìm cột trống đầu tiên để bắt đầu chèn , update col_index khi chèn 
            col_index = 0
            while col_index < num_cols and row[col_index] is not None:
                col_index += 1

            for seg in segments:
                if seg == '':
                    # empty cell => treat as empty string
                    rowspan, colspan, data = 1, 1, ''
                else:
                    rowspan, colspan, data = parse_span(seg)
                
                cell_value = clean_wiki_markup(data)
                # Tìm col_index khả dụng tiếp theo trong trường hợp cột trước đã bị chiếm bởi rowspan/colspan.
                while col_index < num_cols and row[col_index] is not None:
                    col_index += 1
                if col_index >= num_cols:
                    # no space left: skip (or break)
                    break
                # gán giá trị cho cell, theo colspan
                for i in range(colspan):
                    if col_index + i < num_cols:
                        row[col_index + i] = cell_value
                # nếu rowspan > 1 -> lưu lại để áp dụng cho các hàng tiếp theo, remaining rows = rowspan - 1
                if rowspan > 1:
                    rowspan_tracker[(current_row_index, col_index)] = (rowspan - 1, cell_value)
                #tăng các col index qa các cell đã đặt 
                col_index += colspan
            continue

        continue

        
            
    
    # --- Tạo DataFrame ---
    # bỏ các hàng toàn None
    table_rows = [r for r in table_rows if not all(v is None for v in r)]
    df = pd.DataFrame(table_rows, columns=headers)
    return df

# --- Sử dụng ---

#test 
b0 ="Trấn Thành"
b1 = "Ninh Dương Lan Ngọc"
b2 = "Ngô Thanh Vân"
a0 = b2
a1 = (extract_films_section(get_wikitext(a0)))
a2= extract_films_from_subsection(a1)
# print_json_format(a2)

# Tự lấy headers
headers = extract_headers(a2[0]['subsection_wikitext'])
print("Headers:", headers)

# Parse table
df = wikitable_to_df(a2[0]['subsection_wikitext'], headers)
print(df)
save_to_txt(str(df), auto_name = True)


Headers: ['Năm', 'Tựa đề', 'Đạo diễn', 'Sản xuất', 'Vai diễn', 'Ghi chú']
     Năm                                  Tựa đề Đạo diễn Sản xuất  \
0   2001                                Hương dẻ    Không    Không   
1   2004                                   Rouge    Không    Không   
2   2006                          Ngôi nhà bí ẩn    Không    Không   
3   2006                               2 trong 1    Không    Không   
4   2006                     Chuyện tình Sài Gòn    Không    Không   
5   2007                       Dòng máu anh hùng    Không    Không   
6   2009                                Bẫy rồng    Không    Không   
7   2010                       Biệt đội ưng biển    Không    Không   
8   2011                      Khao khát đỉnh cao    Không       Có   
9   2012                          Ngọc viễn đông    Không    Không   
10  2012                      Ngôi nhà trong hẻm    Không    Không   
11  2012                              Hành trình    Không    Không   
12  2013        

## extract_films_from_person_wikitext

In [320]:
def extract_films_from_person_wikitext(wikitext):
    '''
    wikitext - all
    wikitext - section chứa mọi phim
    wikitext - section từng loại phim 
        [
            {
                "title": "Phim điện ảnh",
                "subsection_wikitext": "{| class=\"wikitable\"\n|- style=\"background:#ccc; text-align:center;\"\n! rowspan=\"2\" style=\"width:33px;\"|Năm\n! rowspan=\"2\" | Tựa đề\n! colspan=\"2\" |Vai trò\n! rowspan=\"2\" |Vai diễn\n! rowspan=\"2\" |Ghi chú\n|-\n! Đạo diễn\n! Sản xuất\n|-\n|2001\n|''Hương dẻ''\n| {{Không}}\n| {{Không}}\n|Na lớn\n|rowspan=\"2\" |phim truyền hình\n|-\n|2004\n|''Rouge''\n| {{Không}}\n| {{Không}}\n|Thủy\n|-\n| rowspan=\"3\" | 2006\n|''Ngôi nhà bí ẩn''\n| {{Không}}\n| {{Không}}\n|Đạo diễn Trúc\n|\n|-\n|''2 trong 1''\n| {{Không}}\n| {{Không}}\n|Như Lan\n| \n|-\n|''Chuyện tình Sài Gòn''\n| {{Không}}\n| {{Không}}\n|Tâm\n|2008 chiếu ở Việt Nam\n|-\n|2007\n|''[[Dòng máu anh hùng]]''\n| {{Không}}\n| {{Không}}\n|Võ Thanh Thúy\n|\n|-\n|2009\n|''[[Bẫy rồng]]''\n| {{Không}}\n| {{Không}}\n|Trinh / Phượng hoàng\n|\n|-\n|2010\n|''Biệt đội ưng biển''\n| {{Không}}\n| {{Không}}\n|Nữ cảnh sát đa quốc gia Tú Mi\n|\n|-\n|2011\n|''Khao khát đỉnh cao''\n| {{Không}}\n| {{Có}}\n|Ngô Thanh Vân\n|phim tài liệu của [[365daband|365]] (nhóm nhạc do cô thành lập)\n|-\n| rowspan=\"3\" |2012\n|''Ngọc viễn đông''\n| {{Không}}\n| {{Không}}\n|Huyên\n|\n|-\n|''[[Ngôi nhà trong hẻm]]''\n| {{Không}}\n| {{Không}}\n|Thảo\n|-\n|''Hành trình''\n| {{Không}}\n| {{Không}}\n|Người chị\n|Phim ngắn\n|-\n|2013\n|''[[Lửa Phật]]''\n| {{Không}}\n| {{Không}}\n|Ánh\n|\n|-\n|2015\n|''Ngày nảy ngày nay''\n| {{Không}}\n| {{Có}}\n|Đan Nương\n|\n|-\n| rowspan=\"3\" |2016\n|''[[Siêu trộm (phim)|Siêu trộm]]''\n| {{Không}}\n| {{Không}}\n|Kỳ\n|Vai diễn khách mời\n|-\n|''Ngọa hổ tàng long 2''\n| {{Không}}\n| {{Không}}\n|Mantis\n|\n|-\n|''[[Tấm Cám: Chuyện chưa kể]]''\n| {{Có}}\n| {{Có}}\n|Dì ghẻ\n|\n|-\n| rowspan=\"4\" |2017\n|''[[Blade & Soul|Blade & Soul: Sứ mệnh người được chọn]]''\n| {{Không}}\n| {{Không}}\n|Sát thủ\n|Phim ngắn\n|-\n|''[[Cô Ba Sài Gòn]]''\n| {{Không}}\n| {{Có}}\n|Bà Thanh Mai\n|\n|-\n|''[[Star Wars: Jedi cuối cùng]]''\n| {{Không}}\n| {{Không}}\n|Paige Tico\n|\n|-\n|''[[Chiếc đũa quyền năng]]''\n| {{Không}}\n| {{Không}}\n|Tien\n|\n|-\n| rowspan=\"2\" |2018\n|''[[Về quê ăn Tết]]''\n| {{Không}}\n| {{Có}}\n|Đậu Xanh\n|\n|-\n|''[[Song lang (phim)|Song lang]]''\n| {{Không}}\n| {{Có}}\n|\n|\n|-\n|2019\n|''[[Hai Phượng]]''\n| {{Không}}\n| {{Có}}\n|Hai Phượng\n|\n|-\n| rowspan=\"3\" |2020\n|''Chim ưng đen''\n| {{Không}}\n| {{Có}}\n| Điệp vụ Hương\n| Phim ngắn\n|-\n|''[[Năm chiến hữu]]''\n| {{Không}}\n| {{Không}}\n|Hanoi Hannah\n|\n|-\n|''[[The Old Guard: Những chiến binh bất tử]]''\n| {{Không}}\n| {{Không}}\n|Quỳnh\n|\n|-\n|2021\n|''[[Trạng Tí phiêu lưu ký]]''\n| {{Không}}\n| {{Có}}\n|\n|\n|-\n| rowspan=\"2\" |2022\n|''The Princess''\n| {{không}}\n| {{không}}\n| Linh\n|\n|-\n|''[[Thanh Sói – Cúc dại trong đêm]]''\n| {{Có}}\n| {{Có}}\n|Dì Lin \"Jacqueline\"\n|\n|-\n|2023\n| ''[[Kẻ kiến tạo]]''\n| {{Không}}\n| {{Không}}\n| Kami\n|\n|-\n|2025\n| ''The Old Guard 2''\n| {{Không}}\n| {{Không}}\n| Quỳnh\n|\n|-\n|}"
            },
            {
                "title": "Chương trình truyền hình",
                "subsection_wikitext": "* 2009: ''[[Hành trình kết nối những trái tim]]''\n* 2010: ''[[Bước nhảy hoàn vũ (2010)|Bước nhảy hoàn vũ]]''\n* 2012, 2013: ''[[Thử thách cùng bước nhảy (chương trình truyền hình)|Thử thách cùng bước nhảy]]'' (giám khảo)\n* 2013: ''[[Project Runway Vietnam: Nhà thiết kế Thời trang Việt Nam|Project Runway Vietnam]]'' (dẫn chương trình)"
            },
            {
                "title": "Diễn xuất trong MV",
                "subsection_wikitext": "* Awakening ([[365daband|365]])\n* Ác mộng ([[365daband|365]])\n* Chia xa ([[Tuấn Hưng]])\n* Dạ khúc cho tình nhân ([[Đàm Vĩnh Hưng]])\n* Dẫu tình đã xa ([[Minh Thuận]]) \n* Hai cô tiên ([[365daband|365]]) (cảnh trong phim ''Ngày nảy ngày nay'')\n* Hai cô tiên (Dance version) ([[365daband|365]])\n* Merry Christmas ([[365daband|365]])\n* Mưa (Avi Kim Anh)\n* Ngọc Lan ([[Trần Thu Hà]])\n* Về quê ăn Tết ([[Jun Phạm]])"
            }
        ]
    
    '''
    
    all_films_section = extract_films_section(wikitext)
    
    films_from_subsection_dict = extract_films_from_subsection(all_films_section)
    films_by_type = {}  # dict kết quả cuối cùng

    for subsection in films_from_subsection_dict:
        type_name = subsection["title"]  # ví dụ: "Phim điện ảnh"
        sub_wikitext = subsection["subsection_wikitext"].strip()

        # --- Nếu không phải bảng wikitable thì bỏ qua ---
        if not sub_wikitext.startswith("{|"):
            print(f"Bỏ qua '{type_name}' (không phải bảng wikitable)")
            continue

        # --- Xử lý bảng wikitable ---
        headers = extract_headers(sub_wikitext)
        df_films = wikitable_to_df(sub_wikitext, headers)
        films_dict = df_to_dict(df_films)

        # --- Ghi vào dict kết quả ---
        films_by_type[type_name] = films_dict
        

    return films_by_type

#test 
# print_json_format(extract_films_from_person_wikitext(get_wikitext("Trấn Thành")))
print_json_format(extract_films_from_person_wikitext(ndln_wikitext))

{
    "Truyền hình": [
        {
            "Năm": "2009",
            "Tựa phim": "Gia đình phép thuật",
            "Vai diễn": "Phương",
            "Kênh": "HTV7"
        },
        {
            "Năm": "2009",
            "Tựa phim": "Những mảnh vỡ phù hoa",
            "Vai diễn": "",
            "Kênh": "HTV7"
        },
        {
            "Năm": "2010",
            "Tựa phim": "Mệnh lệnh Hoa hồng",
            "Vai diễn": "Bích Thủy",
            "Kênh": "HTV7"
        },
        {
            "Năm": "2012",
            "Tựa phim": "Phiên chợ số",
            "Vai diễn": "Thảo",
            "Kênh": "HTV9"
        },
        {
            "Năm": "2014",
            "Tựa phim": "Vừa đi vừa khóc",
            "Vai diễn": "Lụa",
            "Kênh": "VTV3"
        },
        {
            "Năm": "2014",
            "Tựa phim": "Sóng gió hôn nhân",
            "Vai diễn": "Ánh Giao",
            "Kênh": "HTV9"
        },
        {
            "Năm": "2016",
            "Tựa phim"

In [321]:
def get_all_films_from_films_dict(films_dict):
    all_films = []
    for type_name, films in films_dict.items():
        for film in films:
            # thêm thông tin loại phim để dễ phân biệt
            film_with_type = film.copy()
            film_with_type["Thể loại"] = type_name
            all_films.append(film_with_type)
    return all_films

#test
z0 = extract_films_from_person_wikitext(get_wikitext("Trấn Thành"))
z1 = get_all_films_from_films_dict(z0)

for f in z1:
    print(f)

{'Năm': '2006', 'Tựa': 'Mùi ngò gai', 'Vai': 'MC', 'Chú thích': '', 'Thể loại': 'Phim truyền hình'}
{'Năm': '2007', 'Tựa': 'Đời cần có nhau', 'Vai': 'Bá', 'Chú thích': '', 'Thể loại': 'Phim truyền hình'}
{'Năm': '2009', 'Tựa': 'Cô nàng bướng bỉnh', 'Vai': 'Hải Long', 'Chú thích': '', 'Thể loại': 'Phim truyền hình'}
{'Năm': '2009', 'Tựa': 'Ngõ vắng', 'Vai': 'Vũ', 'Chú thích': '', 'Thể loại': 'Phim truyền hình'}
{'Năm': '2009', 'Tựa': 'Sau ánh hào quang', 'Vai': 'bác sĩ Vũ', 'Chú thích': '', 'Thể loại': 'Phim truyền hình'}
{'Năm': '2010', 'Tựa': 'Bước chân hoàn vũ', 'Vai': 'MC', 'Chú thích': '', 'Thể loại': 'Phim truyền hình'}
{'Năm': '2010', 'Tựa': 'Cổng mặt trời (phim truyền hình)', 'Vai': 'Bạn của Quốc', 'Chú thích': '', 'Thể loại': 'Phim truyền hình'}
{'Năm': '2010', 'Tựa': 'Phía sau ánh bình minh', 'Vai': 'Việt Hoàn', 'Chú thích': '', 'Thể loại': 'Phim truyền hình'}
{'Năm': '2010', 'Tựa': 'Chuyện tình mùa thu', 'Vai': 'Bảo', 'Chú thích': '', 'Thể loại': 'Phim truyền hình'}
{'Năm': '

## extract_cast_section

In [37]:
def extract_cast_section(wikitext):
    text_lower = wikitext.lower()
    
    # == dien vien == , ==dien vien== 
    keywords = [
        "== diễn viên ==",
        "==diễn viên==",
    ]

    # tìm vị trí tiêu đề '== diễn viên =='
    start_position = -1
    for k in keywords: 
        pos = text_lower.find(k)
        if pos != -1:
            start_position = pos
            key = k
            break

    
    
    if start_position == -1: return "" # str rỗng, ko có 

    # cắt phần vb sau title dvien
    content = wikitext[start_position + len(key):] 

    # tìm vị trí tiêu đề tiếp theo (== ... ==)
    end_position = content.find("==")
    if end_position != -1: section = content[:end_position] 
    else: section = content # k có title next -> lay het 

    return section.strip() # delete space thừa


print(extract_cast_section(matBiec_wikitext))

[[Tập tin:Mắt biếc – Behind the scenes (2).png|nhỏ|Dàn diễn viên phim ''Mắt biếc'']]
*[[Trần Nghĩa (diễn viên)|Trần Nghĩa]] vai Ngạn:<ref>{{Chú thích web|url=http://baodatviet.vn/van-hoa/tin-tuc-giai-tri/victor-vu-truy-lung-nang-mat-biec-va-chang-si-tinh-3368589/|title=Victor Vũ truy lùng nàng Mắt Biếc và chàng si tình|last=|first=|date=5 tháng 11 năm 2018|website=Báo Đất Việt|archive-url=https://web.archive.org/web/20190716065358/http://baodatviet.vn/van-hoa/tin-tuc-giai-tri/victor-vu-truy-lung-nang-mat-biec-va-chang-si-tinh-3368589/|archive-date=2019-07-16|url-status=dead|access-date=20 tháng 1 năm 2019}}</ref><ref>{{chú thích web|url=https://ione.vnexpress.net/tin-tuc/phim/vietnam/vuot-qua-hon-1-400-ung-vien-mat-biec-cua-victor-vu-da-lo-dien-3886413.html|website=IONE.VNEXPRESS.NET|tiêu đề=Vượt qua hơn 1.400 ứng viên, 'Mắt Biếc' của Victor Vũ đã lộ diện|ngày truy cập=2019-02-27|ngày=2019-02-26|ngôn ngữ=vi}}</ref> Chàng trai sinh ra và lớn lên ở làng Đo Đo, mang dáng vẻ thư sinh cùng 

In [38]:
#test

nhaBaNu = get_wikitext("Nhà bà Nữ")

## extract_cast_list

In [39]:
import re
def extract_cast_list(wikitext):
    wikicode = mwparserfromhell.parse(wikitext)
    cast_list = []

    # lay all dòng begin * :*[[Trúc Anh]]
    lines = []
    # duyệt từng node => chuyển node thành chuỗi, bỏ khoảng trắng thừa => if line begin * -> add vào lines

    # default 
    actor_name_list = []
    character_list = []
    
    for node in wikicode.nodes:  
        # bỏ qa mọi dạng template {{...}}
        if node.__class__.__name__ == "Template": continue

        text = str(node).strip()  

        if ("*" in text) or (not text):
            continue
        
        # lấy những line begin [[ , vai 
        # [[Trúc
        # vai Hà Lan
        if text.startswith('[[Tập tin:'): continue       
        if text.startswith('[['):       
            if ("[[" in text) and ("]]" in text):
                text = text.replace("[[", "").replace("]]", "")
                if "|" in text:
                    text = text.split("|")[0].strip()      
            actor_name_list.append(text)

        if (text.startswith('vai')) or (text.startswith('trong vai')):
          
            match = re.search(r"vai\s+([^,.:]+)", text)
          
            if match: 
                text = match.group(1).strip()
                # if res begin "trong", bỏ trong
                if text.startswith("trong "):
                    text = text[6:].strip()

                character_list.append(text)
            else: print("Không tìm thấy vai")


    for actor, character in zip(actor_name_list, character_list):
        actor_obj = {
            "actor_name": actor,
            "character": character
        }
        # add list
        cast_list.append(actor_obj)

    
    return cast_list


#test 

print(extract_cast_list(extract_cast_section(matBiec_wikitext)))
print(len(extract_cast_list(extract_cast_section(matBiec_wikitext))))



[{'actor_name': 'Trần Nghĩa (diễn viên)', 'character': 'Ngạn'}, {'actor_name': 'Trúc Anh', 'character': 'Hà Lan'}, {'actor_name': 'Khánh Huyền', 'character': 'Cô của Hà Lan'}, {'actor_name': 'Thanh Hiền', 'character': 'Bà nội của Ngạn'}]
4


In [40]:
#test 

print(extract_cast_list(extract_cast_section(nhaBaNu)))

[{'actor_name': 'Lê Giang', 'character': 'Ngọc Nữ'}, {'actor_name': 'Uyển Ân', 'character': 'Ngọc Nhi'}, {'actor_name': 'Song Luân', 'character': 'John'}, {'actor_name': 'Trấn Thành', 'character': 'Phú Nhuận'}, {'actor_name': 'Khả Như', 'character': 'Ngọc Như'}, {'actor_name': 'Ngọc Giàu', 'character': 'Ngọc Ngà'}, {'actor_name': 'Lê Dương Bảo Lâm', 'character': 'Lê Minh Hổ'}, {'actor_name': 'Việt Anh (nghệ sĩ)', 'character': 'ông Liêm'}, {'actor_name': 'Công Ninh', 'character': 'bố của John'}, {'actor_name': 'Ngân Quỳnh', 'character': 'mẹ của John'}]


[[Tập tin:Mắt biếc – Behind the scenes (2).png|nhỏ|Dàn diễn viên phim ''Mắt biếc'']] 
 
* 
[[Trần Nghĩa (diễn viên)|Trần Nghĩa]] 
vai Ngạn: 

## extract_actors_from_film

In [41]:
def extract_actors_from_film(wikitext):
    # Trích danh sách diễn viên từ '== Diễn viên ==' của wikitext phim
    cast_section = extract_cast_section(wikitext)
    
    #list 
    cast_list = extract_cast_list(cast_section)
    # {name: , char: }


    vn_cast_list = []
    for cast in cast_list:
        actor_name = cast.get('actor_name')  # lấy tên diễn viên
        if not actor_name: continue  # bỏ qua 

        if is_vn_person_wikilink(actor_name):
            vn_cast_list.append(cast)
        
    return vn_cast_list

#test
matBiec_actors = extract_actors_from_film(matBiec_wikitext)

print(matBiec_actors)

[{'actor_name': 'Trần Nghĩa (diễn viên)', 'character': 'Ngạn'}, {'actor_name': 'Trúc Anh', 'character': 'Hà Lan'}, {'actor_name': 'Khánh Huyền', 'character': 'Cô của Hà Lan'}, {'actor_name': 'Thanh Hiền', 'character': 'Bà nội của Ngạn'}]


## extract_directors_from_film

In [42]:
def extract_directors_from_film(wikitext):
    infobox = extract_infobox_from_wikitext(wikitext)
    director = infobox.get('đạo diễn')
    if ("[[" in director) and ("]]" in director):
        director = director.replace("[[", "").replace("]]", "")

    return director
#test
print(extract_directors_from_film(matBiec_wikitext))

Victor Vũ


In [43]:
SEED_LIST = read_seeds(SEED_LIST_FILE)

print(SEED_LIST)

['Trấn Thành', 'Ninh Dương Lan Ngọc', 'Ngô Thanh Vân', 'Hồng Đào', 'Kiều Minh Tuấn', 'Victor Vũ', 'Charlie Nguyễn', 'Vũ Ngọc Đãng', 'Kaity Nguyễn', 'Jun Vũ']


In [44]:
print(extract_actors_from_film(get_wikitext("Dòng máu anh hùng")))

[{'actor_name': 'Johnny Trí Nguyễn', 'character': 'Lê Văn Cường'}, {'actor_name': 'Ngô Thanh Vân', 'character': 'Sỹ'}, {'actor_name': 'Dustin Nguyễn', 'character': 'Tham Nguyễn'}, {'actor_name': 'Nguyễn Chánh Tín', 'character': 'Hứa Danh'}, {'actor_name': 'Nguyễn Thắng', 'character': 'Sĩ quan Tessier'}, {'actor_name': 'Stephane Gauger', 'character': 'Người gián điệp'}, {'actor_name': 'Nguyễn Hậu', 'character': 'Công nhân trong mỏ sắt'}]


In [45]:
print(extract_actors_from_film(get_wikitext("Bộ tứ báo thủ")))

[{'actor_name': 'Trần Tiểu Vy', 'character': 'Quỳnh Anh'}, {'actor_name': 'Nguyễn Cao Kỳ Duyên (người mẫu)', 'character': 'Karen'}, {'actor_name': 'Trấn Thành', 'character': 'cậu Mười Một'}, {'actor_name': 'Lê Dương Bảo Lâm', 'character': 'Jessica (Chét-Xi-Cà)'}, {'actor_name': 'Lê Giang', 'character': 'dì Bốn'}, {'actor_name': 'Uyển Ân', 'character': 'Kiều'}, {'actor_name': 'Ngọc Giàu', 'character': 'chủ phòng trọ Jessica (Chét-Xi-Cà)'}, {'actor_name': 'Việt Anh (nghệ sĩ)', 'character': 'sếp Quốc Anh'}, {'actor_name': 'Quân A.P', 'character': 'chàng trai của Quỳnh Anh'}]


In [322]:
z2 = extract_films_from_person_wikitext(get_wikitext("Ngô Thanh Vân"))
print(get_all_films_from_films_dict(z2))

Bỏ qua 'Chương trình truyền hình' (không phải bảng wikitable)
Bỏ qua 'Diễn xuất trong MV' (không phải bảng wikitable)
[{'Năm': '2001', 'Tựa đề': 'Hương dẻ', 'Đạo diễn': 'Không', 'Sản xuất': 'Không', 'Vai diễn': 'Na lớn', 'Ghi chú': 'phim truyền hình', 'Thể loại': 'Phim điện ảnh'}, {'Năm': '2004', 'Tựa đề': 'Rouge', 'Đạo diễn': 'Không', 'Sản xuất': 'Không', 'Vai diễn': 'Thủy', 'Ghi chú': 'phim truyền hình', 'Thể loại': 'Phim điện ảnh'}, {'Năm': '2006', 'Tựa đề': 'Ngôi nhà bí ẩn', 'Đạo diễn': 'Không', 'Sản xuất': 'Không', 'Vai diễn': 'Đạo diễn Trúc', 'Ghi chú': '', 'Thể loại': 'Phim điện ảnh'}, {'Năm': '2006', 'Tựa đề': '2 trong 1', 'Đạo diễn': 'Không', 'Sản xuất': 'Không', 'Vai diễn': 'Như Lan', 'Ghi chú': '', 'Thể loại': 'Phim điện ảnh'}, {'Năm': '2006', 'Tựa đề': 'Chuyện tình Sài Gòn', 'Đạo diễn': 'Không', 'Sản xuất': 'Không', 'Vai diễn': 'Tâm', 'Ghi chú': '2008 chiếu ở Việt Nam', 'Thể loại': 'Phim điện ảnh'}, {'Năm': '2007', 'Tựa đề': 'Dòng máu anh hùng', 'Đạo diễn': 'Không', 'Sản xu